In [ ]:
import os
import sys
import datetime
import time
import urllib.request
import json

def get_request_url(url) :
    client_id="O4NyArTHoAMfTbatmh_V"
    client_secret="u2lqH1a89B"
    print(url)
    rep=urllib.request.Request(url)
    print(rep.get_header)
    rep.add_header("X-Naver-Client-Id",client_id)
    rep.add_header("X-Naver-Client-Secret",client_secret)
    
    try :
        response=urllib.request.urlopen(rep)
        if response.getcode() == 200 :
            print("[%s] Url request success" % datetime.datetime.now())
            return response.read().decode("utf-8")
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

def getNaverSearchResult(sNode, search_text, page_start, display) :
    base="https://openapi.naver.com/v1/search"
    node="/%s.json" % sNode
    params="?query=%s&start=%s&display=%s" % (urllib.parse.quote(search_text),
                                              page_start, display)
    url=base+node+params
    
    getData=get_request_url(url)
    
    if (getData == None) :
        return None
    else :
        return json.loads(getData)

def getPosrData(post, jsonResult) :
    title=post["title"]
    description=post["description"]
    link=post["link"]
    org_link=post["originallink"]
    
    pDate=datetime.datetime.strptime(post["pubDate"], 
                                    "%a, %d %b %Y %H:%M:%S +0900")
    pDate=pDate.strftime("%Y-%m-%d %H:%M:%S")
    jsonResult.append({"title":title, "description":description,
                      "org_link":org_link, "link":link, "pDate":pDate})
    
    return

def main() :
    jsonResult=[]
    sNode="news" # blog , cafearticle
    search_text="파이썬"
    display_count=100
    page_start=1
    
    jsonSearch=getNaverSearchResult(sNode, search_text, page_start, display_count)
    with open("Raw_%s_naver_%s.json" % (search_text, sNode), "w", encoding="utf-8" ) as outfile :
        retJson=json.dumps(jsonSearch,indent=4,sort_keys=True,ensure_ascii=False)
        outfile.write(retJson)
    
    
    while ((jsonSearch != None) and (jsonSearch["display"]!=0)) :
        for post in jsonSearch["items"] :
            getPosrData(post, jsonResult)
        nStart=jsonSearch["start"]+jsonSearch["display"]
        jsonSearch=getNaverSearchResult(sNode, search_text, nStart, display_count)
    
    with open("%s_naver_%s.json" % (search_text, sNode), "w", encoding="utf-8" ) as outfile :
        retJson=json.dumps(jsonResult,indent=4,sort_keys=True,ensure_ascii=False)
        outfile.write(retJson)
    print("%s_naver_%s.json SAVED" % (search_text, sNode))
    
if __name__ == "__main__" :
    main() 